<a href="https://colab.research.google.com/github/amin-kd/People-Counting-using-wifi-probe-requests/blob/main/peopleCounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install tshark
!pip install pyshark

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libbcg729-0 libc-ares2 liblua5.2-0 libnl-genl-3-200 libpcap0.8 libsbc1
  libsmi2ldbl libspandsp2 libspeexdsp1 libwireshark-data libwireshark15
  libwiretap12 libwsutil13 wireshark-common
Suggested packages:
  snmp-mibs-downloader geoipupdate geoip-database geoip-database-extra
  libjs-leaflet libjs-leaflet.markercluster wireshark-doc
The following NEW packages will be installed:
  libbcg729-0 libc-ares2 liblua5.2-0 libnl-genl-3-200 libpcap0.8 libsbc1
  libsmi2ldbl libspandsp2 libspeexdsp1 libwireshark-data libwireshark15
  libwiretap12 libwsutil13 tshark wireshark-common
0 upgraded, 15 newly installed, 0 to remove and 35 not upgraded.
Need to get 23.0 MB of archives.
After this operation, 120 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1

In [ ]:
import os
import nest_asyncio
import pyshark
from pyshark import FileCapture as pyshark_FileCapture
import pandas as pd
import numpy as np

In [ ]:
nest_asyncio.apply() #enable async I/O on colab

In [43]:
probeRequests = pyshark_FileCapture('test1.pcapng', display_filter='wlan.fc.type_subtype == 0x04')

In [120]:
feature_vectors = []


for frame in probeRequests:
  try:
    layer = frame.layers
    timestamp = float(frame.sniff_timestamp)
    sa = layer[2].sa
    rssi = layer[1].signal_dbm
    dataRate = layer[1].data_rate
    channel = layer[1].channel
    frequency = layer[1].frequency
    vendorOUI = sa[:8].upper()
    firstByte = int(vendorOUI[0:2], 16)
    windowId = int(timestamp) // 300 # for 5min intervals
    # bitwise operation to check local admin bit.
    LG = (firstByte & 0b10) == 0                 # True: Real Mac                                 False: Locally Administered (randomized)
    IG = (firstByte & 0b01) == 0                 # True: Unicast                                  False: Multicast/Broadcast

    ssid_temp = layer[3].wlan_ssid if hasattr(layer[3], 'wlan_ssid') else None
    ssid = ssid_temp if ssid_temp != 'SSID: ' else None

    feature_vector = {'timestamp': timestamp,'sa': sa, 'rssi': rssi, 'dataRate': dataRate,'ssid':ssid, 'frequency': frequency, 'channel': channel, 'LG': LG, 'IG': IG, 'windowId': windowId}
    feature_vectors.append(feature_vector)

  except (AttributeError, IndexError, ValueError) as e:
    print(f"frame cannot be processed: {e}")
    continue

In [121]:
print(feature_vectors)

[{'timestamp': 1751730347.389034, 'sa': 'a6:f6:0d:ca:4f:a2', 'rssi': '-85', 'dataRate': '5.5', 'ssid': None, 'frequency': '2412', 'channel': '1', 'LG': False, 'IG': True, 'windowId': 5839101}, {'timestamp': 1751730347.409142, 'sa': 'a6:f6:0d:ca:4f:a2', 'rssi': '-86', 'dataRate': '1', 'ssid': None, 'frequency': '2412', 'channel': '1', 'LG': False, 'IG': True, 'windowId': 5839101}, {'timestamp': 1751730347.429252, 'sa': 'a6:f6:0d:ca:4f:a2', 'rssi': '-85', 'dataRate': '1', 'ssid': None, 'frequency': '2412', 'channel': '1', 'LG': False, 'IG': True, 'windowId': 5839101}, {'timestamp': 1751730348.117669, 'sa': '04:0e:3c:6c:9d:97', 'rssi': '-90', 'dataRate': '1', 'ssid': 'TIM-27186405', 'frequency': '2412', 'channel': '1', 'LG': True, 'IG': True, 'windowId': 5839101}, {'timestamp': 1751730348.127811, 'sa': '04:0e:3c:6c:9d:97', 'rssi': '-91', 'dataRate': '1', 'ssid': 'TIM-27186405', 'frequency': '2412', 'channel': '1', 'LG': True, 'IG': True, 'windowId': 5839101}, {'timestamp': 1751730348.2344